## Use of Plotly to compare different algorithms

How to interact with the data:
- There are button on the top right corner whenever you hover over the graph.
- Click on the names in the legend to make that data appear or disappear.
- Drag the axis in the middle to move right or left.
- Drag the axis from the very end to change the interval of that axis. 
- Use the dropdown to view the different singers. 

In [ ]:
import plotly.offline
import plotly.express as px
import plotly.io as pio
from plotly import tools
plotly.offline.init_notebook_mode(connected=True) 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from copy import deepcopy
from plotly.offline import init_notebook_mode, iplot, plot
import numpy as np

### Scherbaum Mshavanadze

In [ ]:
data = {
    "boersma": {
        "GVM009": {},
        "GVM017": {},
        "GVM019": {},
        "GVM031": {},
        "GVM097": {}
    },
    "noll": {
        "GVM009": {},
        "GVM017": {},
        "GVM019": {},
        "GVM031": {},
        "GVM097": {}
    },
    "praat": {
        "GVM009": {},
        "GVM017": {},
        "GVM019": {},
        "GVM031": {},
        "GVM097": {}
    },
    #"yin": {
    #    "GVM009": {},
    #    "GVM017": {},
    #    "GVM019": {},
    #    "GVM031": {},
    #    "GVM097": {}
    #},
    "crepe": {
        "GVM009": {},
        "GVM017": {},
        "GVM019": {},
        "GVM031": {},
        "GVM097": {}
    },
    "maddox": {
        "GVM009": {},
        "GVM017": {},
        "GVM019": {},
        "GVM031": {},
        "GVM097": {}
    },
    "hermes": {
        "GVM009": {},
        "GVM017": {},
        "GVM019": {},
        "GVM031": {},
        "GVM097": {}
    }
}


def separate(adir, algo):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)

data_dir = "/Akamai/voice/data/pitches/"
#data_dir_raw = "/Akamai/voice/data/pitches-raw/"

for algorithm in sorted(os.listdir(data_dir)):
    if algorithm in data:
        for collection in sorted(os.listdir(f"{data_dir}{algorithm}")):
            if collection != "Scherbaum Mshavanadze":
                continue
            for song in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}")):
                for location in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}/{song}")):
                    if (location[-4:] == '.txt'):
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{location}", algo=algorithm)
                        #xRaw, yRaw = separate(f"{data_dir_raw}{algorithm}/{collection}/{song}/{location}", algo=algorithm)
                        
                        data[algorithm][song[:6]][location[-7:-4]] = (x, y)  
                        #dataRaw[algorithm][song[:6]][location[-7:-4]] = (xRaw, yRaw)  

In [ ]:
def graph (song):
    pio.templates.default = "plotly_dark"
    
    fig = make_subplots(rows=1, cols=1)   
    
    
    with open(r'/Akamai/voice/notebooks/pitch_detection/matlabOutput/GVM009single.txt') as f:
        a = f.read().splitlines()

    person=[0] * 10
    person[0] = list()   
    person[1] = list()   
    person[2] = list()   
    person[3] = list()   
    person[4] = list()
    person[5] = list()
    person[6] = list()

    for row in a:
        rowArray = row.split(',')
        for i in range(0,len(rowArray)):
            person[i].append(rowArray[i])
        for i in range(len(rowArray),7):
            person[i].append(0)
    
    fig.append_trace(
                    go.Scattergl(
                        x = person[0],
                        y = person[1],
                        name="MATLAB:person1",
                        mode="markers",
                        visible=True
                    ), 1, 1
                )
    
    for key, val in data.items():
        for location, dat in data[key][song].items():
            if (location[0] == 'S' or location[0] == 'X') and location[1] != '5':
                v = True if location[1] == '1' else False
            
                        
                fig.append_trace(
                    go.Scattergl(
                        x = data[key][song][location][0],
                        y = data[key][song][location][1],
                        name=f"{key}: {location}",
                        mode="markers",
                        visible=v
                    ), 1, 1
                )
    
#    for key, val in dataRaw.items():
#        for location, dat in dataRaw[key][song].items():
#            if (location[0] == 'S' or location[0] == 'X') and location[1] != '5':
#                v = True if location[1] == '1' else False
#            
##                        
#                fig.append_trace(
#                    go.Scattergl(
#                        x = dataRaw[key][song][location][0],
#                        y = dataRaw[key][song][location][1],
#                        name=f"{key}-raw: {location}",
#                       mode="markers",
#                        visible=v
#                    ), 1, 1
#                )
#     for key, val in dataRaw.items():
#         for location, dat in dataRaw[key][song].items():
#             if (location[0] == 'S' or location[0] == 'X') and location[1] != '5':
#                 v = True if location[1] == '1' else False
            
                        
#                 fig.append_trace(
#                     go.Scattergl(
#                         x = dataRaw[key][song][location][0],
#                         y = abs(dataRaw[key][song][location][1]-data[key][song][location][1]),
#                         name=f"{key}-raw: {location}",
#                         mode="markers",
#                         visible=v
#                     ), 1, 1
#                 )

    buttons = []
    labels = ['Person 1', 'Person 2', 'Person 3']
    for i, label in enumerate(labels):
        visibility = [i==j for j in range(len(labels))]
        print(visibility)
        button = dict(
            label = label,
            method = "update",
            args = [{ 'visible': visibility}]
        )

        buttons.append(button)

    updatemenus = list([
        dict(active=0,
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x = 0.005,
            y = 1.06,
        )
    ])
   
    fig.update_layout(
        legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
        ),
        margin=dict(l=0, r=0, t=100, b=0)
    )
    

    fig.update_traces(
        marker=dict(size=3),
        selector=dict(mode='markers')
    )
    

    fig['layout']['title'] = song
    fig['layout']['width'] = 900
    fig['layout']['height'] = 500
    fig['layout']['showlegend'] = True
    fig['layout']['updatemenus'] = updatemenus

    iplot(fig, filename="dropdown")

In [ ]:
graph('GVM009')

In [ ]:
graph('GVM017')

In [ ]:
graph('GVM017')

In [ ]:
graph('GVM019')

In [ ]:
graph('GVM031')

In [ ]:
graph('GVM097')

### Teach Yourself Megrelian Songs

In [ ]:
data2 = {}

directories = ["Ak'a Si Rekisho",
               "Gepshvat Ghvini", 
               "Io _ Chkin Kiana", 
               "Mesishi Vardi",
               "Meureme", 
               "Mi Re Sotsodali_",
               "Mole Chit'i Gilakhe",
               "O Da", 
               "Vojanudi Chkim Jargvals"]

algos = ['boersma', 'noll', 'crepe', 'maddox', 'hermes'] # Note: later add praat and yin

for algo in algos:
    data2[algo] = {}
    for direct in directories:
        data2[algo][direct] = {
            "mix": {},
            "bass": {},
            "middle": {},
            "top": {}
        }


def separate(adir, algo):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)

data_dir = "/Akamai/voice/data/pitches/"

for algorithm in sorted(os.listdir(data_dir)):
    if algorithm in data2:
        for collection in sorted(os.listdir(f"{data_dir}{algorithm}")):
            if collection != "Teach Yourself Megrelian Songs":
                continue
            for song in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}")):
                for location in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}/{song}")):
                    if (location[-4:] == '.txt'):
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{location}", algo=algorithm)
                        
                        if not 'AHDS' in location:
                            data2[algorithm][song]['mix'] = (x, y)
                        else:
                            name_tag = location[(location.index('AHDS') + 3):(location.index('AHDS') + 6)]
                            
                            if name_tag[1] == '1':
                                data2[algorithm][song]['bass'] = (x, y)
                            elif name_tag[1] == '2':
                                data2[algorithm][song]['middle'] = (x, y)
                            else:
                                data2[algorithm][song]['top'] = (x, y)

In [ ]:
def graph2 (song):
    pio.templates.default = "plotly_dark"
    
    fig = make_subplots(rows=1, cols=1)
    
    type_list = []
    for algo, collection in data2.items():
        for name, part in collection.items():
            for audio_type, res in part.items():
                if name != song:
                    continue
                
                try:
                    fig.append_trace(
                            go.Scattergl(
                                x = res[0],
                                y = res[1],
                                name=f"{algo}: {audio_type}",
                                mode="markers",
                                visible= (True if audio_type == "bass" else False)
                            ), 1, 1
                        )
                    
                    type_list.append(audio_type)
                except:
                    print(f"{algo}: {audio_type} not available")
    
    fig.update_layout(
        legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
        ),
        margin=dict(l=0, r=0, t=100, b=0)
    )
    
    buttons = []
    labels = ['bass', 'middle', 'top', 'mix']
    for i, label in enumerate(labels):
        visibility = [label==current_type for current_type in type_list]
        button = dict(
            label = label,
            method = "update",
            args = [{ 'visible': visibility}]
        )

        buttons.append(button)
    
    updatemenus = list([
        dict(active=0,
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x = 0.005,
            y = 1.06,
        )
    ])
    

    fig.update_traces(
        marker=dict(size=3),
        selector=dict(mode='markers')
    )
    

    fig['layout']['title'] = song
    fig['layout']['width'] = 900
    fig['layout']['height'] = 500
    fig['layout']['showlegend'] = True
    fig['layout']['updatemenus'] = updatemenus

    iplot(fig, filename="dropdown")

In [ ]:
graph2("Ak'a Si Rekisho")

In [ ]:
graph2("Gepshvat Ghvini")

In [ ]:
graph2("Io _ Chkin Kiana")

In [ ]:
graph2("Mesishi Vardi")

In [ ]:
graph2("Meureme")

In [ ]:
graph2("Mi Re Sotsodali_")

In [ ]:
graph2("Mole Chit'i Gilakhe")

In [ ]:
graph2("O Da")

In [ ]:
graph2("Vojanudi Chkim Jargvals")

### Teach Yourself Gurian Songs

In [ ]:
data2 = {}

directories = ["Ak'a Si Rekisho",
               "Gepshvat Ghvini", 
               "Io _ Chkin Kiana", 
               "Mesishi Vardi",
               "Meureme", 
               "Mi Re Sotsodali_",
               "Mole Chit'i Gilakhe",
               "O Da", 
               "Vojanudi Chkim Jargvals"]

algos = ['boersma', 'noll', 'crepe', 'maddox', 'hermes'] # Note: later add praat and yin

for algo in algos:
    data2[algo] = {}
    for direct in directories:
        data2[algo][direct] = {
            "mix": {},
            "bass": {},
            "middle": {},
            "top": {}
        }


def separate(adir, algo):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)

data_dir = "/Akamai/voice/data/pitches/"

for algorithm in sorted(os.listdir(data_dir)):
    if algorithm in data2:
        for collection in sorted(os.listdir(f"{data_dir}{algorithm}")):
            if collection != "Teach Yourself Gurian Songs":
                continue
            for song in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}")):
                for location in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}/{song}")):
                    if (location[-4:] == '.txt'):
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{location}", algo=algorithm)
                        
                        if not 'AHDS' in location:
                            data2[algorithm][song]['mix'] = (x, y)
                        else:
                            name_tag = location[(location.index('AHDS') + 3):(location.index('AHDS') + 6)]
                            
                            if name_tag[1] == '1':
                                data2[algorithm][song]['bass'] = (x, y)
                            elif name_tag[1] == '2':
                                data2[algorithm][song]['middle'] = (x, y)
                            else:
                                data2[algorithm][song]['top'] = (x, y)

### Atalay's Code

In [ ]:
def confidenceInterval(song,algorithm,compare = ""):
    print("sum of squares of", algorithm)
    print("song:", song)
    if compare != "":
        print("ground truth creator algorithm:", compare)
    else:
        compare = algorithm
    for key, val in data.items():
        if algorithm == key:
            ground = dict()
            model = dict()
            for location, dat in data[key][song].items():
                if (location[0] == 'S' or location[0] == 'X') and location[1] != '5':
                    if(location[0] == 'S'):
                        model[location[1]] = data[key][song][location][1]
                    if(location[0] == 'X'):
                        ground[location[1]] = data[compare][song][location][1]
            diff = 0
            for key in ground:
                diff += np.sum((ground[key]-model[key])**2)
            
            print("diff" , round(diff))
            print('---------')

def confidenceIntervalCompareBest(song,algorithm):
    print("sum of squares of", algorithm)
    print("song:", song)
    smallest=float("inf")
    bestAlgo = ""
    for key, val in data.items():
            print(key)
            ground = dict()
            model = dict()
            for location, dat in data[key][song].items():
                if (location[0] == 'S' or location[0] == 'X') and location[1] != '5':
                    if(location[0] == 'S'):
                        model[location[1]] = data[key][song][location][1]
                    if(location[0] == 'X'):
                        ground[location[1]] = data[key][song][location][1]
            
            diff = 0
            print(ground)
            print(model)
            for key in ground:
                diff += np.sum((ground[key]-model[key])**2)
            if(diff < smallest):
                smallest = diff
                bestAlgo = key
                
    print("best algorithm:", bestAlgo)
            
    for key, val in data.items():
        if algorithm == key:
            ground = dict()
            model = dict()
            for location, dat in data[key][song].items():
                if (location[0] == 'S' or location[0] == 'X') and location[1] != '5':
                    if(location[0] == 'S'):
                        model[location[1]] = data[key][song][location][1]
                    if(location[0] == 'X'):
                        ground[location[1]] = data[bestAlgo][song][location][1]
            
            diff = 0
            for key in ground:
                diff += np.sum((ground[key]-model[key])**2)
            
            print("diff" , round(diff))
            print('---------')
            
# confidenceIntervalCompareBest("GVM009", "boersma")